In [3]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion_model")
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced


2023-02-01 18:51:02 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [4]:
importlib.reload(w2vU)
importlib.reload(lds)
importlib.reload(ced)
#
# #tess = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device="cpu")
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                             directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                             directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                             directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                             device="cpu")
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list)

In [12]:
# maxnum = 0
# maxnum1 = 0
# maxnum2 = 0
# maxindex = 0
# for i in range(0, len(emo_dataset) - 1):
#     lo = emo_dataset[i][0].cpu()
#
#     #size = (processor(lo)["input_values"][0][0].shape)[0]
#     size = lo.shape[1]
#     maxnum = max(maxnum, size)
#     if(maxnum != size):
#         maxnum1 = max(maxnum1, size)
#     if(maxnum != size and maxnum1 != size):
#         maxnum2 = max(maxnum2, size)
#     if(maxnum == size):
#         maxindex = i


In [15]:
# print(100000 / 16000)
# print(maxnum1 / 16000)
# print(maxnum2 / 16000)

6.25
7.531125
7.2345


In [5]:
# f,s = torchaudio.load(emo_dataset.dataFrame["path"][4640])
# len(f[0])
# lo = emo_dataset[6000][0].cpu()
# len(lo[0])
# processor(lo, return_tensors="pt")
# print(processor(lo[0], return_tensors="pt"), lo[0])

In [5]:
import datasets

# importlib.reload(lds)
#
# loaded = emo_dataset[500][0].cpu()
# samplerate = emo_dataset.samplerate
#
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
#res = torchaudio.transforms.Resample(orig_freq=samplerate, new_freq=sr)(loaded)
#output = processor(res)
#processor.pad()

## Debug old model

In [10]:
import network_models.w2v_emotion_model.custom_model as cm
import  network_models.w2v_emotion_model.custom_collator as cc
import gc
import importlib
device = "cpu"
#importlib.reload(cm)
# import network_models.soundstream_lstm.model_trainer as ct
#
#model = cm.W2V_EmotionClassifierSevenEmos().to(device=device)
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

gc.collect()
importlib.reload(cm)
#tess = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess")

model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device).to("cpu")
model2 = cm.W2V_EmotionClassifierSevenEmos().to(device=device)

In [11]:
from torch.utils.data import DataLoader

data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list)).collate_fn
#loader = DataLoader(newSet, shuffle=True, num_workers=2 ,collate_fn=data_collator)
arr = np.asarray(newSet[0])
vals = data_collator(arr[None, :])

#vals = enumerate(loader)[0]
model(vals[0])

<generator object Module.parameters at 0x7f762b84a6c0>


/tmp/ipykernel_517752/824063702.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asarray(newSet[0])


tensor([[0.3689, 0.0625, 0.3361, 0.0012, 0.1861, 0.0440, 0.0012]],
       grad_fn=<SoftmaxBackward0>)

In [46]:
from transformers import Wav2Vec2PreTrainedModel, Wav2Vec2Model, Wav2Vec2Config


config = Wav2Vec2Config(name_or_path=model_name_or_path)
wav2vec2 = Wav2Vec2Model(config).to(device)
def gen(data):
    outputs = wav2vec2(data)
    return torch.mean(outputs[0], dim=1)

In [57]:
enc = gen(vals[0])
#model2(enc)

tensor([[-0.9008, -0.3096,  0.7530,  0.1439,  0.7899,  0.3786, -0.2956,  0.4463,
         -0.7978,  0.4299, -0.4700, -0.2694,  0.4110, -0.8196, -0.5054,  0.0825,
          0.6343, -1.6062, -0.3723, -0.2128, -0.6787, -0.6757, -0.5471,  0.0829,
          0.9843, -0.1209,  0.7534,  0.2680,  0.2844,  0.8103, -0.6863,  0.2118,
         -0.0171,  1.2855, -1.9144, -0.5711,  0.0812,  0.4870, -0.9308,  0.3020,
          0.0284,  0.5335,  0.1846,  0.7637,  0.4484, -0.5277, -0.4951,  0.5641,
          0.7578, -0.0168, -0.1442,  0.3758, -0.2371,  0.1367,  1.2959,  0.0593,
         -0.4436, -1.4407,  0.2164, -0.8998,  0.0868, -0.8606,  0.0490,  0.3743,
         -0.5595, -1.6432,  0.2320,  0.7943,  0.8324,  0.2879, -0.0803, -0.4916,
         -0.5705,  0.5791,  0.1537, -0.0113, -0.9397, -0.1012, -1.2689, -0.7750,
          0.4140, -0.1385, -0.7271,  0.0056,  0.2039,  0.7528,  0.1659, -0.4618,
         -0.6326, -0.8201,  0.2963,  0.1479,  0.3667, -0.5136, -0.4786, -0.5223,
         -0.2813, -0.6745,  

# Training old Model

In [9]:
# importlib.reload(ADU)
# trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)

In [10]:
# emo_dataset.label_list

In [10]:
# importlib.reload(ct)
# trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=8, save_model_every=500, num_epochs=5001, model_path="content/classifier/W2VClassifier/Nr1/")

In [11]:
# import gc
# gc.collect()
#
# trainer.train()

# Evaluation

In [2]:
importlib.reload(w2vU)
importlib.reload(lds)
importlib.reload(ced)
device = "cpu"
# emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
#                                              directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
#                                              directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
#                                              directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
#                                              device="cpu")
emo_dataset = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device=device)
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
#processor.to(device)
#
#torch.save(model.state_dict(), "content/classifier/W2VClassifier/Nr1/model_3363.pth")

newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)

In [3]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2Config, Wav2Vec2Model
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.w2v_emotion_model.custom_model as cm
import gc
import network_models.soundstream_lstm.model_trainer as ct
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

gc.collect()
importlib.reload(cm)
#tess = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess")

model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device)
# # config
# config = AutoConfig.from_pretrained(
#     model_name_or_path,
#     num_labels=len(tess.label_list),
#     label2id={label: i for i, label in enumerate(tess.label_list)},
#     id2label={i: label for i, label in enumerate(tess.label_list)},
#     finetuning_task="wav2vec2_clf",
# )
# setattr(config, 'pooling_mode', pooling_mode)
#
# processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
# target_sampling_rate = processor.feature_extractor.sampling_rate
#
# conf = Wav2Vec2Config(name_or_path=model_name_or_path)
# model = Wav2Vec2Model(conf)

#model(tess[0][0], tess[0][1])

In [7]:
newSet[0]

(array([ 0.00907809, -0.08372961, -0.13303371, ...,  0.0525817 ,
         0.04243086,  0.03082989], dtype=float32),
 array([1]))

In [4]:
import  network_models.w2v_emotion_model.custom_collator as cc
import  network_models.w2v_emotion_model.custom_trainer as ctct
from transformers import TrainingArguments, EvalPrediction
trainSet, evalSet = ADU.train_val_dataset(newSet, 0.02)
trainSet, evalSet = ADU.train_val_dataset(evalSet, 0.1)

importlib.reload(ctct)
importlib.reload(cm)
importlib.reload(cc)

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == np.argmax(p.label_ids, axis=1)).astype(np.float32).mean().item()}

model.freeze_feature_extractor()
data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))
# training_args = TrainingArguments(
#     output_dir="content/W2VClassifier/Nr2",
#     # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=2,
#     evaluation_strategy="steps",
#     num_train_epochs=10.0,
#     fp16=True,
#     save_steps=100,
#     eval_steps=100,
#     logging_steps=100,
#     learning_rate=1e-4,
#     save_total_limit=2,
# )

# trainer = ctct.CTCTrainer(
#     model=model,
#     data_collator=data_collator,
#     args=training_args,
#     compute_metrics=compute_metrics,
#     train_dataset=trainSet,
#     eval_dataset=evalSet,
#     tokenizer=processor.feature_extractor,
# )

import network_models.soundstream_lstm.model_trainer as ct1


# importlib.reload(ct1)
# trainer = ct1.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=8, save_model_every=500, num_epochs=5001, model_path="content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)

In [ ]:
importlib.reload(ct1)
trainer = ct1.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=2, save_model_every=500, num_epochs=5001, model_path="content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
gc.collect()
trainer.train()


Epoch 1
-------------------------------
loss: 1.945971  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.945413 

Epoch 2
-------------------------------
loss: 1.942434  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.944779 

Epoch 3
-------------------------------
loss: 1.945729  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.944705 

Epoch 4
-------------------------------
loss: 1.937100  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.944342 

Epoch 5
-------------------------------
loss: 1.907578  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.943351 

Epoch 6
-------------------------------
loss: 1.960479  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.943211 

Epoch 7
-------------------------------
loss: 1.954976  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.942728 

Epoch 8
-------------------------------
loss: 1.962167  [    0/   50]
Test Error: 
 Accuracy: 16.7%, Avg loss: 1.942149 

Epoch 9
----------------

Epoch 1
-------------------------------
[2]
[5]


TypeError: cannot unpack non-iterable NoneType object